In [2]:
%load_ext autoreload
%autoreload 2

import pathlib
import psycopg2
import psycopg2.extras
import pandas as pd

from create_table import init_database_and_tables

from sql_queries import (
    review_check,
    calendar_check,
    listings_check,
    reviewer_check,
    time_check
)

psycopg2.__version__

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'2.9.3 (dt dec pq3 ext lo64)'

In [3]:
# import numpy as np
# from psycopg2.extensions import register_adapter, AsIs

# def addapt_numpy_array(numpy_array):
#     return AsIs(tuple(numpy_array))

# register_adapter(np.ndarray, addapt_numpy_array)

In [4]:
init_database_and_tables()

In [5]:
conn = psycopg2.connect("host=172.21.0.2 dbname=airbnbdb port=5432 user=airbnb password=airbnb")

print("PostgreSQL server information")
print(conn.get_dsn_parameters(), "\n")

cur = conn.cursor()

PostgreSQL server information
{'user': 'airbnb', 'channel_binding': 'prefer', 'dbname': 'airbnbdb', 'host': '172.21.0.2', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [6]:
def execute_values(conn, df, table):
    """
    Using psycopg2.extras.execute_values() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s ON CONFLICT DO NOTHING" % (table, cols)
    cursor = conn.cursor()
    try:
        psycopg2.extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()

# Merge Data

In [7]:
listings_March = pd.read_csv("../../data/listings.csv")
listings_June = pd.read_csv("../../data/listings_1.csv")
listings_September = pd.read_csv("../../data/listings_2.csv")
listings_December = pd.read_csv("../../data/listings_3.csv")

calendar_March = pd.read_csv("../../data/calendar.csv")
calendar_June = pd.read_csv("../../data/calendar_1.csv")
calendar_September = pd.read_csv("../../data/calendar_2.csv")
calendar_December = pd.read_csv("../../data/calendar_3.csv")

reivew_March = pd.read_csv("../../data/reviews.csv")
reivew_June = pd.read_csv("../../data/reviews_1.csv")
reivew_September = pd.read_csv("../../data/reviews_2.csv")
reivew_December = pd.read_csv("../../data/reviews_3.csv")


listings_list = [listings_March, listings_June, listings_September, listings_December]
calendar_list = [calendar_March, calendar_June, calendar_September, calendar_December]
review_list = [reivew_March, reivew_June, reivew_September, reivew_December]

listings = pd.concat(listings_list)
calendar = pd.concat(calendar_list)
review = pd.concat(review_list)

In [8]:
listings = listings.sample(n=50)
calendar = calendar.sample(n=50)
review = review.sample(n=50)

In [9]:
print("Shape of listings df:", listings.shape)
print("Shape of calendar df:", calendar.shape)
print("Shape of reviews df:", review.shape)

Shape of listings df: (50, 74)
Shape of calendar df: (50, 7)
Shape of reviews df: (50, 6)


# Reviewer table

In [10]:
reviewer_table = review[["reviewer_id", "reviewer_name"]]
reviewer_table

,reviewer_id,reviewer_name
257579,96553947,Callie
397905,7847454,Inna
228921,30899246,Rosanne
134403,42497908,Ronnie
20689,6172922,Violaine
320837,112907476,Iris
387127,7144280,Juul
316732,54785592,Océane
130863,133409981,Joshua
204974,73678451,Shannon


In [11]:
reviewer_table.isna().sum()

reviewer_id      0
reviewer_name    0
dtype: int64

In [12]:
execute_values(conn, reviewer_table, table='reviewer')

execute_values() done


In [13]:
cur.execute(reviewer_check)

for row in cur.fetchall():
    print(row)

(96553947, 'Callie')
(7847454, 'Inna')
(30899246, 'Rosanne')
(42497908, 'Ronnie')
(6172922, 'Violaine')


# Time table

In [14]:
time_table = calendar[["date"]]
time_table["date"] = pd.to_datetime(time_table.date, format='%Y-%m-%d')

/tmp/ipykernel_17248/1529482266.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_table["date"] = pd.to_datetime(time_table.date, format='%Y-%m-%d')


In [15]:
t = time_table['date']
dt = t.dt

In [16]:
time_data = (t, dt.day, dt.month, dt.year, dt.isocalendar().week, dt.weekday)
column_labels = ("date_time", "day", "month", "year", "week","dayofweek")

In [17]:
time_df = pd.DataFrame({k: v for k, v in zip(column_labels, time_data)})
time_df.head()

,date_time,day,month,year,week,dayofweek
1847982,2022-04-12,12,4,2022,15,1
3376195,2022-01-24,24,1,2022,4,0
3403531,2021-07-14,14,7,2021,28,2
5248394,2022-08-16,16,8,2022,33,1
352340,2021-07-31,31,7,2021,30,5


In [18]:
execute_values(conn, time_df, table='time')

execute_values() done


In [19]:
cur.execute(time_check)

for row in cur.fetchall():
    print(row)

(datetime.datetime(2022, 4, 12, 0, 0), 12, 4, 2022, 15, 1)
(datetime.datetime(2022, 1, 24, 0, 0), 24, 1, 2022, 4, 0)
(datetime.datetime(2021, 7, 14, 0, 0), 14, 7, 2021, 28, 2)
(datetime.datetime(2022, 8, 16, 0, 0), 16, 8, 2022, 33, 1)
(datetime.datetime(2021, 7, 31, 0, 0), 31, 7, 2021, 30, 5)


# Listings

In [20]:
col = [ 'id',
        'name' ,
        'room_type',
        'number_of_reviews' ,
        'review_scores_rating',
        'review_scores_value',
        'host_id' ,
        'host_name' ,
        'host_is_superhost',
        'host_listings_count' ,
        'neighbourhood',
        'property_type' ,
        'accommodates' ,
        'availability_30' ,
        'bathrooms_text',
        'bedrooms' ,
        'beds',
        'price' ,
        'minimum_nights',
        'maximum_nights',
        ]

listings_table = listings[col]

In [21]:
# Importing regex
import re

bath = ["bathrooms_text"]
for col in bath:
    listings_table[col] = listings_table[col].apply(lambda x: str(re.compile('[a-zA-Z]').sub('', str(x))))
    listings_table[col] = listings_table[col].apply(lambda x: str(re.compile(' ').sub('', str(x))))

/tmp/ipykernel_17248/3008764699.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings_table[col] = listings_table[col].apply(lambda x: str(re.compile('[a-zA-Z]').sub('', str(x))))
/tmp/ipykernel_17248/3008764699.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings_table[col] = listings_table[col].apply(lambda x: str(re.compile(' ').sub('', str(x))))


In [22]:
listings_table["bathrooms"] = pd.to_numeric(listings_table["bathrooms_text"],errors='coerce')
listings_table.drop(['bathrooms_text'], axis=1, inplace=True)

/tmp/ipykernel_17248/4152102839.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings_table["bathrooms"] = pd.to_numeric(listings_table["bathrooms_text"],errors='coerce')
/tmp/ipykernel_17248/4152102839.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings_table.drop(['bathrooms_text'], axis=1, inplace=True)


In [23]:
# The price fields in our data frame

listings_table['price'] = listings_table['price'].str.replace(',', '')
listings_table['price'] = listings_table['price'].str.replace('$', '')
listings_table['price'] = listings_table['price'].astype(float)

/tmp/ipykernel_17248/4006336846.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings_table['price'] = listings_table['price'].str.replace(',', '')
/tmp/ipykernel_17248/4006336846.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  listings_table['price'] = listings_table['price'].str.replace('$', '')
/tmp/ipykernel_17248/4006336846.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [24]:
for col in ['bathrooms', 'bedrooms', 'beds']:
    listings[col].fillna(listings[col].median(), inplace=True)

/home/flynn/.cache/pypoetry/virtualenvs/airbnb-EHIZReGs-py3.9/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1216: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [25]:
listings_table.dtypes

id                        int64
name                     object
room_type                object
number_of_reviews         int64
review_scores_rating    float64
review_scores_value     float64
host_id                   int64
host_name                object
host_is_superhost        object
host_listings_count     float64
neighbourhood            object
property_type            object
accommodates              int64
availability_30           int64
bedrooms                float64
beds                    float64
price                   float64
minimum_nights            int64
maximum_nights            int64
bathrooms               float64
dtype: object

In [26]:
execute_values(conn, listings_table, table='listings')

execute_values() done


In [27]:
cur.execute(listings_check)

for row in cur.fetchall():
    print(row)

(18820779, 'Fascinating private canal view loft', 'Entire home/apt', 0, Decimal('NaN'), Decimal('NaN'), 130896439, 'Emma', 'f', Decimal('5.0'), 'Amsterdam, Noord-Holland, Netherlands', 'Entire apartment', 2, 0, Decimal('NaN'), Decimal('1.0'), Decimal('250.0'), 7, 1125, Decimal('1.0'))
(41334300, 'Ground Floor Apartment in the heart of Amsterdam', 'Entire home/apt', 3, Decimal('5.0'), Decimal('5.0'), 22318017, 'Bella', 'f', Decimal('0.0'), 'Amsterdam, Noord-Holland, Netherlands', 'Entire rental unit', 4, 5, Decimal('2.0'), Decimal('2.0'), Decimal('350.0'), 2, 1125, Decimal('1.0'))
(13964167, 'Cosy appartment 100m2 in quiet area of Amsterdam', 'Entire home/apt', 1, Decimal('100.0'), Decimal('10.0'), 83223419, 'Jelle', 'f', Decimal('1.0'), 'Amsterdam, Noord-Holland, Netherlands', 'Entire townhouse', 3, 0, Decimal('2.0'), Decimal('3.0'), Decimal('99.0'), 3, 10, Decimal('1.0'))
(35859555, 'Spacious 4P Nassau canal apartment', 'Private room', 8, Decimal('4.5'), Decimal('4.13'), 269726966, 'A

# Calendar

In [28]:
col = ['listing_id',
        'date',
        'available',
        'adjusted_price']

calendar_table = calendar[col]

In [29]:
calendar['adjusted_price'] = calendar['adjusted_price'].str.replace(',', '')
calendar['adjusted_price'] = calendar['adjusted_price'].str.replace('$', '')
calendar['price'] = calendar['adjusted_price'].astype(float)

/tmp/ipykernel_17248/2756646630.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  calendar['adjusted_price'] = calendar['adjusted_price'].str.replace('$', '')


In [30]:
calendar_table = calendar[["listing_id","date","available","price"]]

In [31]:
calendar_table.head()

,listing_id,date,available,price
1847982,50950057,2022-04-12,f,70.0
3376195,20486092,2022-01-24,f,130.0
3403531,19399581,2021-07-14,f,135.0
5248394,40894292,2022-08-16,f,105.0
352340,1954182,2021-07-31,t,300.0


In [32]:
execute_values(conn, calendar_table, table='calendar')

execute_values() done


In [33]:
cur.execute(calendar_check)

for row in cur.fetchall():
    print(row)

(50950057, datetime.datetime(2022, 4, 12, 0, 0), 'f', Decimal('70.0'))
(20486092, datetime.datetime(2022, 1, 24, 0, 0), 'f', Decimal('130.0'))
(19399581, datetime.datetime(2021, 7, 14, 0, 0), 'f', Decimal('135.0'))
(40894292, datetime.datetime(2022, 8, 16, 0, 0), 'f', Decimal('105.0'))
(1954182, datetime.datetime(2021, 7, 31, 0, 0), 't', Decimal('300.0'))


# Review

In [34]:
col = ['listing_id',
        'id',
        'date',
        'comments']

review_table = review[col]

In [35]:
review_table = review_table.rename(columns={ 'id': 'review_id' })
                                           

In [36]:
review_table.head()

,listing_id,review_id,date,comments
257579,14996718,149850986,2017-05-06,Staying at Lara and Damir's apartment was perf...
397905,28307853,352428628,2018-11-25,Nice location.
228921,12897197,169354089,2017-07-12,Mark and Andrea’s apartment was a wonderful ho...
134403,5207981,45864093,2015-09-06,Carlijn and Joep were simply amazing! This was...
20689,608432,13065701,2014-05-19,L'appartement de Gordon et Amy est extrêmement...


In [37]:
execute_values(conn, review_table, table='review')

execute_values() done


In [38]:
cur.execute(review_check)

for row in cur.fetchall():
    print(row)

(14996718, 149850986, datetime.datetime(2017, 5, 6, 0, 0), "Staying at Lara and Damir's apartment was perfect for my friend's and mine trip to Amsterdam. Even though they were in Croatia when we arrived, a very kind friend of theirs met us at the apartment to let us in. The walk to the city center was beautiful and the market next to the apartment was a great way to spend a chunk of the day! I would recommend Lara and Damir's apartment to anyone and their hospitality was above and beyond!! ")
(28307853, 352428628, datetime.datetime(2018, 11, 25, 0, 0), 'Nice location.')
(12897197, 169354089, datetime.datetime(2017, 7, 12, 0, 0), 'Mark and Andrea’s apartment was a wonderful home-away-from-home.  The apartment is comfortable, spacious, very well appointed, and beautiful, filled with gorgeous art and an amazing collection of contemporary fiction.  The kitchen is very well appointed, and the up-to-date appliances are easy to use.  The location is perfect.  It’s steps away from public trans

In [40]:
conn.close ()